DATA EXTRACTION, PREPROCESSING AND CLEANING FOR HOSPITAL LENTGH OF STAY PREDICTION

METHODOLOGY
1. Extract all_data: WP2_df
2. Obtain sequence number of comorbidity per patient: COM_SEQ
3. Include column of number of comorbidity per patient at each record date: COMORBIDITY_COUNT
4. Query data to obtain only In-Hospital Records: WP2_df_PEDW
5. Include additional risk factors and markers for model:\
a.) 'LOS'= Number of Hospital days per admission\
b.) 'NUM_PRVADMISSION_1YR'= Number of previous admissions in 1 year from current admission date\
c.) 'NUM_PRVADMISSION_1YR_COND'= Number of previous admissions per condition in 1 year from current admission date\
d.) 'NUM_PRVADMISSION_3YR'= Number of previous admissions in 3 years from current admission date\
e.) 'NUM_PRVADMISSION_3YR_COND'= Number of previous admissions per condition in 3 years from current admission date\
f.) 'NUM_PRVHOSPITAL_DAYS_1YR'= Total number of cumulative hospital days in the prev 1 year from current admission date \
g.) 'NUM_PRVHOSPITAL_DAYS_3YR'= Total number of cumulative hospital days in the prev 3 years from current admission date \
h.) SMOKING= Binary classification of smoking status as at the time of admission per patient\
i.) ALCOHOL= Binary classification of alcohol status as at the time of admission per patient\
j.) AUTISM= Binary classification of autism per patient\
k.) BEHAVIOURAL DISORDER= Binary classification of behavioural disorder per patient\
l.) MEDICATION= Binary classification of presence of medication as at the time of admission per patient

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cycler
import seaborn as sns
sns.set()
from dateutil.relativedelta import relativedelta
import datetime

In [2]:
import ibm_db
import ibm_db_dbi
import pandas as pd
import os
import requests
import sqlalchemy
import io



import configparser
import warnings
import csv

from datetime import datetime

warnings.filterwarnings("ignore")


LOADING TABLES OF INDIVIDUAL RISK FACTORS

In [ ]:
#LOAD ALL RISK FACTORS FROM GP RECORDS AND/OR HOSPITAL (PEDW) RECORDS


#ALCOHOL
query = 'SELECT * FROM SAILW1375V.'+ 'WP01_PHEN_EXTRACT_WLGP_ALCOHOL'
WLGP_ALCOHOL= pd.read_sql(query,pd_conn)

query = 'SELECT * FROM SAILW1375V.'+ 'WP01_PHEN_EXTRACT_PEDW_ALCOHOL'
PEDW_ALCOHOL= pd.read_sql(query,pd_conn)


#AUTISM
query = 'SELECT * FROM SAILW1375V.'+ 'WP01_PHEN_EXTRACT_WLGP_AUTISM'
WLGP_AUTISM= pd.read_sql(query,pd_conn)

query = 'SELECT * FROM SAILW1375V.'+ 'WP01_PHEN_EXTRACT_PEDW_AUTISM'
PEDW_AUTISM= pd.read_sql(query,pd_conn)

#BEHAVIOURAL DISORDER
query = 'SELECT * FROM SAILW1375V.'+ 'WP01_PHEN_EXTRACT_WLGP_BEHAVIOUR'
WLGP_BEHAVIOUR= pd.read_sql(query,pd_conn)

query = 'SELECT * FROM SAILW1375V.'+ 'WP01_PHEN_EXTRACT_PEDW_BEHAVIOUR'
PEDW_BEHAVIOUR= pd.read_sql(query,pd_conn)

#BMI
query = 'SELECT * FROM SAILW1375V.'+ 'WP01_PHEN_EXTRACT_WLGP_BMI'
WLGP_BMI= pd.read_sql(query,pd_conn)

#SMOKING
query = 'SELECT * FROM SAILW1375V.'+ 'WP01_PHEN_EXTRACT_WLGP_SMOKING'
WLGP_SMOKING= pd.read_sql(query,pd_conn)


#WEIGHTHEIGHTBMI
query = 'SELECT * FROM SAILW1375V.'+ 'WP01_PHEN_EXTRACT_WLGP_WEIGHTHEIGHTBMI'
WLGP_WEIGHTHEIGHTBMI= pd.read_sql(query,pd_conn)

query = 'SELECT * FROM SAILW1375V.'+ 'WP01_PHEN_EXTRACT_PEDW_WEIGHTHEIGHTBMI'
PEDW_WEIGHTHEIGHTBMI= pd.read_sql(query,pd_conn)

#PHYSICALACTIVITY
query = 'SELECT * FROM SAILW1375V.'+ 'WP01_PHEN_EXTRACT_WLGP_PHYSICALACTIVITY'
WLGP_PHYSICALACTIVITY= pd.read_sql(query,pd_conn)


LOAD DATA

In [4]:
#WP2_df= pd.read_csv("all_data0506.csv") WP02_df_NEW
# WP2_df= pd.read_csv("all_data2906.csv") #DATA FOR LOS PAPER

WP2_df= pd.read_csv("all_data3001.csv") # LOAD MOST RECENT DATA

#SET ADMISSION, DISCHARGE, DOD, AND WOB COLUMNS TO DATETIME VARIABLES
WP2_df['DATE']=pd.to_datetime(WP2_df['DATE'])
WP2_df['DISCH_DT']=pd.to_datetime(WP2_df['DISCH_DT'])
WP2_df['DOD']=pd.to_datetime(WP2_df['DOD'])
WP2_df['WOB']=pd.to_datetime(WP2_df['WOB'])
WP2_df['EPI_STR_DT']=pd.to_datetime(WP2_df['EPI_STR_DT'])

In [5]:
#ESTIMATE AGE AT DATE
WP2_df['AGE_AT_DATE']=(WP2_df['DATE']-WP2_df['WOB'])/np.timedelta64(1,'Y')
WP2_df['AGE_AT_DATE']=WP2_df['AGE_AT_DATE'].astype(int)


#ASSIGN AGE GROUPS AT DATE
bins=[0, 20, 30, 40, 50, 60, 70, 80, 105]
labels= ['<20', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
WP2_df['AGEGRP_AT_DATE']=pd.cut(WP2_df.AGE_AT_DATE, bins, labels=labels, include_lowest=True)

In [6]:
#DROPS DUPLICATES IN TOTAL DATA
WP2_df.drop_duplicates(inplace=True)

In [7]:
# VERIFY COLUMNS IN IMPORTED DATASET
WP2_df.columns

Index(['Unnamed: 0', 'ALF_PE', 'DATE', 'DISCH_DT', 'EPI_STR_DT', 'EPI_END_DT',
       'DIAG_NUM', 'DIAG_CODE', 'DIAG_DESC', 'PROV_UNIT_CD', 'CONDITION',
       'CODE_SYSTEM', 'WOB', 'AGE_AT_INDEX_DATE', 'AGE_GRPS_AT_INDEX_DATE',
       'AGE_GRPS_AT_INDEX_DATE_DESC', 'DOD', 'GNDR_CD', 'GNDR_DESC',
       'LSOA2011_CD_AT_INDEX_DATE', 'WIMD2019_QUINTILE_AT_INDEX_DATE',
       'WIMD2019_QUINTILE_AT_INDEX_DATE_DESC',
       'TOWNSEND2011_QUINTILE_AT_INDEX_DATE',
       'TOWNSEND2011_QUINTILE_AT_INDEX_DATE_DESC',
       'ETHN_EC_ONS_DATE_LATEST_CODE', 'ETHN_EC_ONS_DATE_LATEST_DESC',
       'ETHN_EC_NER_DATE_LATEST_CODE', 'ETHN_EC_NER_DATE_LATEST_DESC',
       'AGE_AT_DATE', 'AGEGRP_AT_DATE'],
      dtype='object')

In [ ]:
# MAXIMUM AGE PF PATIENT
WP2_df['AGE_AT_DATE'].max()

In [ ]:
# ADD NEW COLUMN INDICATING MORTALITY OF PATIENT. BASED ON THE PRESENCE OF A VALUE IN THE DOD (DATE OF DEATH) COLUMN
WP2_df['MORTALITY']= np.where(WP2_df['DOD'].isnull(),0,1)
WP2_df

In [133]:
# OBTAIN DATASET OF UNIQUE MALE PATIENTS WITH MORTALITY= 1
MALE_TM=WP2_df.query("MORTALITY==1 and GNDR_DESC=='MALE'").drop_duplicates(subset=['ALF_PE'], ignore_index=True,  keep='last')

# IDENTIFY PATIENTS WITH MISSING WIMD INDEX
MALE_TM['WIMD2019_QUINTILE_AT_INDEX_DATE'].isnull().sum()

In [135]:
# OBTAIN DATASET OF UNIQUE FEMALE PATIENTS WITH MORTALITY= 1

FEMALE_TM=WP2_df.query("MORTALITY==1 and GNDR_DESC=='FEMALE'").drop_duplicates(subset=['ALF_PE'], ignore_index=True,  keep='last')

# IDENTIFY PATIENTS WITH MISSING WIMD INDEX
FEMALE_TM['WIMD2019_QUINTILE_AT_INDEX_DATE'].isnull().sum()

In [ ]:
# OBTAIN COUNT OF UNIQUE PATIENTS WITH HOSPITAL ADMISSION RECORDS
WP2_df.query('CODE_SYSTEM=="ICD10"').ALF_PE.nunique()

CLEANING OF DATA 

In [12]:
#SORT VALUES CHRONOLOGICALLY BASED ON PATIENT ID (ALF_PE), ADMIT PR GP VISIT DATE (DATE) AND EPISODE START DATE FOR HOSPITAL EPISODES(EPI_STR_DT)
WP2_df=WP2_df.sort_values(by=['ALF_PE','DATE','EPI_STR_DT'], ignore_index=True)

In [13]:
#OBTAIN SEQUENCE ORDER OF COMORBIDITIES PER PATIENT BASED ON FIRST DIAGNOSIS
WP2_df['COM_SEQ']=WP2_df.groupby(['ALF_PE'])['CONDITION'].transform(lambda x: pd.factorize(x)[0]+1)

In [ ]:
# TEST OUTPUT OF COM_SEQ VARIABLE FOR A PATIENT
WP2_df[['ALF_PE','DATE','CONDITION','COM_SEQ']].query('ALF_PE==10000XXX')

In [ ]:
# OBTAIN NUMBER OF UNIQUE ADMISSIONS
WP2_df.query('CODE_SYSTEM=="ICD10"').drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True).shape[0]

DROP MISSING VALUES OF HOSPITAL ADMISSION/DISCHARGE DATES

In [ ]:
# OBTAIN COUNTS OF ADMISSIONS WITH MISSING DISCHARGE DATES
WP2_df.query('CODE_SYSTEM=="ICD10"').DISCH_DT.isnull().sum()

In [17]:
# DROP ROWS WITH MISSING DISCHARGE DATES
WP2_df.drop(WP2_df[(WP2_df['CODE_SYSTEM']=='ICD10') & (WP2_df['DISCH_DT'].isnull())].index, inplace=True)

In [19]:
# RESET ROW INDEXES
WP2_df=WP2_df.reset_index(drop=True)

In [22]:
# PUT IN NUMBER OF LONG TERM COMORBIDITIES PER CORRESPONDING PATIENT AT EACH EVENT/ADMISSION DATE.
WP2_df['COMORBIDITY_COUNT']=WP2_df.groupby(['ALF_PE'])['COM_SEQ'].cummax()

In [ ]:
#TEST OUTPUT OF COMORBIDITY_COUNT
WP2_df[['ALF_PE','DATE','CONDITION','COM_SEQ','COMORBIDITY_COUNT']].query('ALF_PE==10000XXX')

INCLUDE COLUMNS TO CATEGORIZE PATIENTS WITH AUTISM

In [151]:
AUTISM_LIST=WLGP_AUTISM.ALF_PE.unique().tolist()+list(set(PEDW_AUTISM.ALF_PE.unique().tolist())-set(WLGP_AUTISM.ALF_PE.unique().tolist()))
WP2_df['AUTISM']=0
WP2_df.loc[WP2_df['ALF_PE'].isin(AUTISM_LIST), 'AUTISM']=1

In [ ]:
#OBTAIN VALUE COUNTS OF PATIENTS WITH AND WITHOUT AUTISM
WP2_df.drop_duplicates(subset=['ALF_PE'], ignore_index=True).AUTISM.value_counts()

INCLUDE COLUMNS TO CATEGORIZE PATIENTS WITH BEHAVIORAL DISORDER

In [153]:
BEHAV_LIST=WLGP_BEHAVIOUR.ALF_PE.unique().tolist()+list(set(PEDW_BEHAVIOUR.ALF_PE.unique().tolist())-set(WLGP_BEHAVIOUR.ALF_PE.unique().tolist()))
WP2_df['BEHAVIOR_DISORDER']=0
WP2_df.loc[WP2_df['ALF_PE'].isin(BEHAV_LIST), 'BEHAVIOR_DISORDER']=1

In [ ]:
#OBTAIN VALUE COUNTS OF PATIENTS WITH AND WITHOUT BEHAVIOR DISORDER
WP2_df.drop_duplicates(subset=['ALF_PE'], ignore_index=True).BEHAVIOR_DISORDER.value_counts()

INCLUDE COLUMNS TO CATEGORIZE PATIENTS WITH HISTORY OF ALCOHOL

In [155]:
#DROP SOME EVENT DESCRIPTIONS CONTAINED IN THE GP RECORDS. THESE VARIABLES ARE NOT INDICATIVE OF A PATIENTS ALCOHOL STATUS
WLGP_ALCOHOL=WLGP_ALCOHOL.drop(WLGP_ALCOHOL[(WLGP_ALCOHOL.EVENT_CD_DESC=='Alcohol consumption')].index)
WLGP_ALCOHOL=WLGP_ALCOHOL.drop(WLGP_ALCOHOL[(WLGP_ALCOHOL.EVENT_CD_DESC=='Alcohol consumption NOS')].index)
WLGP_ALCOHOL=WLGP_ALCOHOL.drop(WLGP_ALCOHOL[(WLGP_ALCOHOL.EVENT_CD_DESC=='Alcohol units per week')].index)

In [ ]:
#OBTAIN NUMBER OF UNIQUE PATIENTS HOW HAVE PROVIDED AT LEAST ONE DECLARATION OF ALCOHOL STATUS
WP2_df_IDLIST=WP2_df.ALF_PE.unique().tolist()
ALCOHOL_IDLIST=set(WLGP_ALCOHOL.ALF_PE.unique().tolist()+ PEDW_ALCOHOL.ALF_PE.unique().tolist())
len(ALCOHOL_IDLIST)

In [ ]:
#NUMBER OF PATIENTS FROM OUR COHORT WHOSE IDs ARE IN THE ALCOHOL_IDLIST LIST
sum(list in WP2_df_IDLIST for list in ALCOHOL_IDLIST)

In [158]:
#SET DATES TO DATETIME TYPE
WLGP_ALCOHOL['EVENT_DT']=pd.to_datetime(WLGP_ALCOHOL['EVENT_DT'])
PEDW_ALCOHOL['ADMIS_DT']=pd.to_datetime(PEDW_ALCOHOL['ADMIS_DT'])

In [159]:
NON_DRINKER=['Teetotaller']
CURRENT_NON_DRINKER=['Current non drinker']
MODERATE_DRINKER=['Ex-light drinker - (1-2u/day)','Ex-trivial drinker (<1u/day)',
                  'Trivial drinker - <1u/day','Social drinker','Alcohol intake within recommended sensible limits',
                  'Light drinker','Moderate drinker']

HEAVY_DRINKER=['Mental and behavioural disorders due to use of alcohol','[X]Mental and behavioural disorders due to use of alcohol: psychotic disorder',
               'Chronic alcoholism in remission','Alcohol-induced chronic pancreatitis',
               '[X]Mental and behavioural disorders due to use of alcohol: harmful use','[X]Mental and behavioural disorders due to use of alcohol',
               'Alcohol withdrawal delirium','[X]Alcohol withdrawal-induced seizure','Chronic alcoholism NOS','Alcoholic hepatitis',
               'Under care of community alcohol team','[X]Mental and behavioural disorders due to use of alcohol: dependence syndrome',
               'Continuous chronic alcoholism','Alcohol dependence syndrome NOS','Alcoholic fatty liver','Very heavy drinker','Chronic alcoholism',
               'Alcoholic liver damage unspecified','Very heavy drinker - >9u/day','Binge drinker','Alcohol intake above recommended sensible limits', 
               'Alcohol abuse monitoring','Hazardous alcohol use','Heavy drinker','Harmful alcohol use','Alcoholic cirrhosis of liver',
               'Alcohol misuse','Acute alcoholic intoxication in alcoholism',]

DRINKER=['Drinks wine','Spirit drinker','Beer drinker','Drinks beer and spirits','Feels should cut down drinking','Declined referral to specialist alcohol treatment service','Alcohol dependence syndrome',
         'Current non drinker','H/O: alcoholism', 'Alcohol withdrawal syndrome',]

FORMER_DRINKER=['Ex-light drinker - (1-2u/day)','Ex-heavy drinker - (7-9u/day)','Ex-very heavy drinker-(>9u/d)',
                'Ex-trivial drinker (<1u/day)']

CLASS_0_ALCOHOL=NON_DRINKER + MODERATE_DRINKER+ CURRENT_NON_DRINKER

In [160]:
# FUNCTION TO CATEGORISE A PATIENTS ALCOHOL STATUS BASED ON PRESENCE OF ANY HISTORY OF DRINKING
def Alcoholhistory(ID,DATE):
    GPpatient_list=WLGP_ALCOHOL.query('ALF_PE==@ID & EVENT_DT<=@DATE').EVENT_CD_DESC.unique().tolist()
    HOSPpatient_list=PEDW_ALCOHOL.query('ALF_PE==@ID & ADMIS_DT<=@DATE').DIAG_CD_1234_DESC.unique().tolist()
    patient_list= GPpatient_list+list(set(HOSPpatient_list)-set(GPpatient_list))
    
    future_GPpatient_list=WLGP_ALCOHOL.query('ALF_PE==@ID & EVENT_DT>@DATE').EVENT_CD_DESC.unique().tolist()
    
    if len(patient_list)==0:
        if len(future_GPpatient_list)!=0 and  future_GPpatient_list[0]=='Teetotaller':
            status=0
        else:
            status='unknown'
    elif len(patient_list)!=0 and all(element in CLASS_0_ALCOHOL for element in patient_list):
        status=0
    else:
        status=1
    return status

In [161]:
DF_CHECK=WP2_df
DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE'], ignore_index=True)
DF_CHECK['ALCOHOL_HISTORY']=DF_CHECK.apply(lambda x: Alcoholhistory(x['ALF_PE'],x['DATE']), axis=1)


In [162]:
WP2_df=WP2_df.sort_values(by=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
DF_CHECK=DF_CHECK.sort_values(by=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
WP2_df=pd.merge(WP2_df,DF_CHECK[['ALF_PE','DATE','ALCOHOL_HISTORY']])

In [ ]:
# OBTAIN COUNTS OF PATIENTS WITH ALCOHOL STATUS=1, 0, AND 'UNKNOWN'
WP2_df.ALCOHOL_HISTORY.value_counts()

In [164]:
#FUNCTION TO IDENTIFY IF A PATIENT HAS AN AVAILABLE ALCOHOL RECORD
def AlcPresence(X):
    mylist=X.ALCOHOL_HISTORY.unique().tolist()
    if all(element in ['unknown'] for element in mylist):
        presAlcRec=0
    else:
        presAlcRec=1
    return pd.Series(presAlcRec, index=X.index)

In [165]:
WP2_df['PresenceAlcRec']=WP2_df.groupby('ALF_PE',group_keys=False).apply(AlcPresence)

In [ ]:
#IDENTIFY NUMBER OF DISTINCT PATIENTS WITH OBTAINABLE ALCOHOL RECORDS AT SOME POINT IN THEIR TIMELINE
WP2_df.query('PresenceAlcRec==1').drop_duplicates(subset=['ALF_PE']).shape[0]

In [167]:
#CHECK NUMBER OF PATIENTS THAT CHANGE ALCOHOL CATEGORY OVER TIME
def ALCChange(X):
    mylist=X.ALCOHOL_HISTORY.unique().tolist()
    if len(mylist)>1:
        alcchange=1
    else:
        alcchange=0
    return pd.Series(alcchange, index=X.index)

WP2_df['PresenceALCchange']=WP2_df.groupby('ALF_PE',group_keys=False).apply(ALCChange)

INCLUDE COLUMNS TO CATEGORIZE PATIENTS WITH HISTORY OF SMOKING

In [ ]:
#OBTAIN NUMBER OF UNIQUE PATIENTS WHO PROVIDED A DECLARATION OF THEIR SMOKING STATUS AT LEAST ONCE.
SMOKING_IDLIST=WLGP_SMOKING.ALF_PE.unique().tolist()
len(SMOKING_IDLIST)

In [ ]:
#NUMBER OF PATIENTS FROM OUR COHORT WHOSE IDs ARE IN THE SMOKING_IDLIST
sum(list in WP2_df_IDLIST for list in SMOKING_IDLIST)

In [171]:
#SET DATES TO DATETIME TYPE
WLGP_SMOKING['EVENT_DT']=pd.to_datetime(WLGP_SMOKING['EVENT_DT'])

In [172]:
#DROP SOME EVENT_DESCCRIPTION WHICH DO NOT POINT TO PRESENCE OR ABSENCE OF SMOKING HISTORY.
VALUES=['Waterpipe tobacco consumption','Family smoking history','Cigar consumption','Cigarette pack-years','Tobacco consumption NOS','Chews tobacco','Cigarette consumption','Tobacco consumption']
WLGP_SMOKING=WLGP_SMOKING[~WLGP_SMOKING['EVENT_CD_DESC'].isin(VALUES)]

In [173]:
#MAKE LIST OF CATEGORIES
NON_SMOKER=['Never smoked tobacco','Current non-smoker']

EX_SMOKER=['Date ceased smoking','Ex roll-up cigarette smoker','Ex cigar smoker','Ex pipe smoker','Ex-very heavy smoker (40+/day)',
           'Ex-cigarette smoker','Ex-heavy smoker (20-39/day)','Ex smoker', 'Current non-smoker','Stopped smoking', 
           'Ex-smoker - amount unknown']

SMOKER=['Smoking status at 52 weeks','Total time smoked','Pipe tobacco consumption','Smoking restarted','Smoking status between 4 and 52 weeks','Smoking status at 4 weeks',
        'Current smoker annual review - enhanced services administration','Minutes from waking to first tobacco consumption',
        'Smoking cessation programme start date','Recently stopped smoking','Smoking free weeks','Smoking reduced',
        'Negotiated date for cessation of smoking','Smoking started','Keeps trying to stop smoking','Failed attempt to stop smoking',
        'Smoking cessation milestones','Not a passive smoker','Ready to stop smoking','Passive smoker','Thinking about stopping smoking',
        'Pipe smoker','Trivial smoker - < 1 cig/day','Cigarette smoker','Current smoker','Rolls own cigarettes','Trying to give up smoking',
        'Not interested in stopping smoking']

SMOK_UNKNOWN=['Refusal to give smoking status','Tobacco consumption unknown']


In [174]:
#CREATE FUNCTION TO CATEGORISE PATIENT'S SMOKING STATUS AT EACH UNIQUE ADMISSION DATE BASED ON AN AVAILABLE HISTORY OF SMOKING.
def Smokinghistory(ID,DATE):
    patient_list=WLGP_SMOKING.query('ALF_PE==@ID & EVENT_DT<=@DATE').EVENT_CD_DESC.unique().tolist()
    future_patient_list=WLGP_SMOKING.query('ALF_PE==@ID & EVENT_DT>@DATE').EVENT_CD_DESC.unique().tolist()
    
    if len(patient_list)==0:
        if len(future_patient_list)!=0 and  future_patient_list[0]=='Never smoked tobacco':
            status=0
        else:
            status='unknown'
    elif len(patient_list)!=0 and all(element in NON_SMOKER for element in patient_list):
        status=0
    elif len(patient_list)!=0 and all(element in SMOK_UNKNOWN for element in patient_list):
        status='unknown'
    else:
        status=1
    return status

In [175]:
DF_CHECK=WP2_df
DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE'], ignore_index=True)
DF_CHECK['SMOKING_HISTORY']=DF_CHECK.apply(lambda x: Smokinghistory(x['ALF_PE'],x['DATE']), axis=1)

In [176]:
WP2_df=pd.merge(WP2_df,DF_CHECK[['ALF_PE','DATE','SMOKING_HISTORY']])

In [177]:
#CHECK NUMBER OF PATIENTS THAT CHANGE CATEGORY
def SMOKChange(X):
    mylist=X.SMOKING_HISTORY.unique().tolist()
    if len(mylist)>1:
        smokchange=1
    else:
        smokchange=0
    return pd.Series(smokchange, index=X.index)

WP2_df['PresenceSMOKchange']=WP2_df.groupby('ALF_PE',group_keys=False).apply(SMOKChange)

In [ ]:
#FUNCTION TO IDENTIFY IF A PATIENT HAS AN AVAILABLE SMOKING RECORD
def SmokPresence(X):
    mylist=X.SMOKING_HISTORY.unique().tolist()
    if all(element in ['unknown'] for element in mylist):
        presSmokRec=0
    else:
        presSmokRec=1
    return pd.Series(presSmokRec, index=X.index)

WP2_df['PresenceSmokRec']=WP2_df.groupby('ALF_PE',group_keys=False).apply(SmokPresence)

#IDENTIFY NUMBER OF DISTINCT PATIENTS WITH OBTAINABLE SMOKING RECORDS AT SOME POINT IN THEIR TIMELINE
WP2_df.query('PresenceSmokRec==1').drop_duplicates(subset=['ALF_PE']).shape[0]

In [ ]:
#IDENTIFY NUMBER OF DISTINCT PATIENTS WITH OBTAINABLE SMOKING RECORDS AT SOME POINT IN THEIR TIMELINE
WP2_df.query('PresenceSmokRec==1').drop_duplicates(subset=['ALF_PE']).shape[0]

INCLUDE COLUMNS TO CATEGORIZE PATIENTS WITH MEDICATIONS

In [181]:
query = 'SELECT * FROM SAILW1375V.'+ 'WP01_PHEN_EXTRACT_WLGP_LISTOFMEDS'
WLGP_MEDS= pd.read_sql(query,pd_conn)

In [182]:
#SET DATES TO DATETIME TYPE
WLGP_MEDS['EVENT_DT']=pd.to_datetime(WLGP_MEDS['EVENT_DT'])

In [ ]:
#OBTAIN NUMBER OF UNIQUE IDS IN SMOKING RISK FACTOR TABLE
MEDS_IDLIST=WLGP_MEDS.ALF_PE.unique().tolist()
len(MEDS_IDLIST)

In [ ]:
#NUMBER OF PATIENTS FROM OUR COHORT WHOSE IDs ARE IN MEDS  RISK FACTORS TABLE
sum(list in WP2_df_IDLIST for list in MEDS_IDLIST)

In [185]:
#GROUP THE MEDS ACCORDING TO THEIR NAMES
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("TEGRETOL", case=False), 'EVENT_CD_DESC']="TEGRETOL"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("EPILIM", case=False), 'EVENT_CD_DESC']="EPILIM"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("LAMOTRIGINE", case=False), 'EVENT_CD_DESC']="LAMOTRIGINE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("LEVETIRACETAM", case=False), 'EVENT_CD_DESC']="LEVETIRACETAM"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("LYRICA", case=False), 'EVENT_CD_DESC']="LYRICA"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("DESITREND", case=False), 'EVENT_CD_DESC']="DESITREND"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("PHENYTOIN", case=False), 'EVENT_CD_DESC']="PHENYTOIN"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("TERIL", case=False), 'EVENT_CD_DESC']="TERIL"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("CARBAMAZEPINE", case=False), 'EVENT_CD_DESC']="CARBAMAZEPINE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("CLOBAZAM", case=False), 'EVENT_CD_DESC']="CLOBAZAM"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("CLONAZEPAM", case=False), 'EVENT_CD_DESC']="CLONAZEPAM"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("SODIUM VALPROATE", case=False), 'EVENT_CD_DESC']="SODIUM VALPROATE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("EPANUTIN", case=False), 'EVENT_CD_DESC']="EPANUTIN"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("GABAPENTIN", case=False), 'EVENT_CD_DESC']="GABAPENTIN"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("TOPIRAMATE", case=False), 'EVENT_CD_DESC']="TOPIRAMATE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("PHENOBARBITAL", case=False), 'EVENT_CD_DESC']="PHENOBARBITAL"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("DEPAKOTE", case=False), 'EVENT_CD_DESC']="DEPAKOTE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("PRIMIDONE", case=False), 'EVENT_CD_DESC']="PRIMIDONE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("PROCHLORPERAZINE", case=False), 'EVENT_CD_DESC']="PROCHLORPERAZINE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("KEPPRA", case=False), 'EVENT_CD_DESC']="KEPPRA"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("LAMICTAL", case=False), 'EVENT_CD_DESC']="LAMICTAL"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("KEPPRA", case=False), 'EVENT_CD_DESC']="KEPPRA"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("VIGABATRIN", case=False), 'EVENT_CD_DESC']="VIGABATRIN"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("PREGABALIN", case=False), 'EVENT_CD_DESC']="PREGABALIN"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("VALPROIC ACID", case=False), 'EVENT_CD_DESC']="VALPROIC ACID"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("MYSOLINE", case=False), 'EVENT_CD_DESC']="MYSOLINE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("ZONISAMIDE ", case=False), 'EVENT_CD_DESC']="ZONISAMIDE "
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("LYRICA", case=False), 'EVENT_CD_DESC']="LYRICA"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("RIVOTRIL", case=False), 'EVENT_CD_DESC']="RIVOTRIL"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("TOPAMAX", case=False), 'EVENT_CD_DESC']="TOPAMAX"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("ACETAZOLAMIDE", case=False), 'EVENT_CD_DESC']="ACETAZOLAMIDE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("LACOSAMIDE", case=False), 'EVENT_CD_DESC']="LACOSAMIDE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("ETHOSUXIMIDE", case=False), 'EVENT_CD_DESC']="ETHOSUXIMIDE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("OXCARBAZEPINE", case=False), 'EVENT_CD_DESC']="OXCARBAZEPINE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("EPISENTA", case=False), 'EVENT_CD_DESC']="EPISENTA"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("ESLICARBAZEPINE ACETATE", case=False), 'EVENT_CD_DESC']="ESLICARBAZEPINE ACETATE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("SABRIL", case=False), 'EVENT_CD_DESC']="SABRIL"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("RUFINAMIDE", case=False), 'EVENT_CD_DESC']="RUFINAMIDE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("PERAMPANEL", case=False), 'EVENT_CD_DESC']="PERAMPANEL"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("ZONEGRAN", case=False), 'EVENT_CD_DESC']="ZONEGRAN"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("ZARONTIN", case=False), 'EVENT_CD_DESC']="ZARONTIN"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("NEURONTIN", case=False), 'EVENT_CD_DESC']="NEURONTIN"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("TRILEPTAL", case=False), 'EVENT_CD_DESC']="TRILEPTAL"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("PERAMPANEL", case=False), 'EVENT_CD_DESC']="PERAMPANEL"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("TIAGABINE", case=False), 'EVENT_CD_DESC']="TIAGABINE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("ORLEPT", case=False), 'EVENT_CD_DESC']="ORLEPT"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("VIMPAT", case=False), 'EVENT_CD_DESC']="VIMPAT"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("TAPCLOB", case=False), 'EVENT_CD_DESC']="TAPCLOB"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("ZEBINIX", case=False), 'EVENT_CD_DESC']="ZEBINIX"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("TAPCLOB", case=False), 'EVENT_CD_DESC']="TAPCLOB"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("FRISIUM", case=False), 'EVENT_CD_DESC']="FRISIUM"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("CARBAGEN SR", case=False), 'EVENT_CD_DESC']="CARBAGEN SR"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("DIAMOX", case=False), 'EVENT_CD_DESC']="DIAMOX"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("EPIMAZ", case=False), 'EVENT_CD_DESC']="EPIMAZ"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("CONVULEX", case=False), 'EVENT_CD_DESC']="CONVULEX"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("RETIGABINE", case=False), 'EVENT_CD_DESC']="RETIGABINE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("FYCOMPA", case=False), 'EVENT_CD_DESC']="FYCOMPA"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("INOVELON", case=False), 'EVENT_CD_DESC']="INOVELON"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("EMESIDE", case=False), 'EVENT_CD_DESC']="EMESIDE"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("GABITRIL", case=False), 'EVENT_CD_DESC']="GABITRIL"
WLGP_MEDS.loc[WLGP_MEDS['EVENT_CD_DESC'].str.contains("PHENOBARBITONE SODIUM", case=False), 'EVENT_CD_DESC']="PHENOBARBITONE SODIUM"


In [ ]:
# OBTAIN COUNTS OF PRESCRIPTIONS PER MEDICATION
WLGP_MEDS.EVENT_CD_DESC.value_counts()

In [188]:
#CATEGORISE PATIENTS WITH A HISTORY OF MEDICATIONS (PSCHOTROPIC)
def MEDhistory(ID,DATE):
    patient_list=WLGP_MEDS.query('ALF_PE==@ID & EVENT_DT<=@DATE').EVENT_CD_DESC.unique().tolist()
    
    if len(patient_list)==0:
        status=0
    
    elif len(patient_list)!=0:
        status=1
    return status

In [189]:
#ADD NEW COLUMN OF MEDICATIONS TO THE DATA
DF_CHECK=WP2_df
DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE'], ignore_index=True)
DF_CHECK['MEDICATIONS']=DF_CHECK.apply(lambda x: MEDhistory(x['ALF_PE'],x['DATE']), axis=1)

WP2_df=pd.merge(WP2_df,DF_CHECK[['ALF_PE','DATE','MEDICATIONS']])

In [193]:
#CHECK NUMBER OF PATIENTS THAT CHANGE CATEGORY
def MEDIChange(X):
    mylist=X.MEDICATIONS.unique().tolist()
    if len(mylist)>1:
        medchange=1
    else:
        medchange=0
    return pd.Series(medchange, index=X.index)

WP2_df['PresenceMEDIchange']=WP2_df.groupby('ALF_PE',group_keys=False).apply(MEDIChange)

INCLUDE COLUMNS TO CATEGORIZE PATIENTS WITH PHYSICAL ACTIVITY

In [195]:
#SET DATES TO DATETIME TYPE
WLGP_PHYSICALACTIVITY['EVENT_DT']=pd.to_datetime(WLGP_PHYSICALACTIVITY['EVENT_DT'])

In [ ]:
#OBTAIN NUMBER OF UNIQUE PATIENTS WITH A DECLARATION OF THEIR PHYSICAL ACTIVITY STATUS
PHYSICAL_IDLIST=WLGP_PHYSICALACTIVITY.ALF_PE.unique().tolist()
len(PHYSICAL_IDLIST)

In [ ]:
#NUMBER OF PATIENTS FROM OUR COHORT WHOSE IDs ARE IN PHYSICAL ACTIVITY  RISK FACTORS TABLE PHYSICAL_IDLIST
sum(list in WP2_df_IDLIST for list in PHYSICAL_IDLIST)

In [198]:
#MAKE LIST OF CATEGORIES
LIMITED=['Enjoys light exercise','Takes inadequate exercise','Aerobic exercise 0 times/week', 'Exercise physically impossible',
        'Aerobic exercise 1 time/week','General practice physical activity questionnaire physical activity index: inactive', 'Avoids even trivial exercise',
        'General practice physical activity questionnaire physical activity index: moderately inactive','Declined referral to physical exercise programme',
        'FITT activity level 0; no moderate or vigorous activity of 20 minutes duration','Anaerobic exercise 1 time/week','Anaerobic exercise 0 times/wk']

REGULAR=['Enjoys moderate exercise','Aerobic exercise 3+ times/week','Aerobic exercise 2 times/week','General practice physical activity questionnaire physical activity index: moderately active',
        'Enjoys heavy exercise','General practice physical activity questionnaire physical activity index: active','Aerobic exercise three times a week',
        'Enjoys intermediate exercise','FITT activity level 1; 1-4 occasions of a mix of moderate and vigorous activity in 4 weeks', 'FITT activity level 2; 5-11 occasions of a mix of moderate and vigorous activity in 4 weeks',
        'FITT activity level 3; 12 or more occasions of moderate activity in 4 weeks','Aerobic exercise four times a week','FITT activity level 4; 12 or more occasions of a mix of moderate and vigorous activity in 4 weeks',
        '30 minutes a day of at least moderate intensity walking on five or more days of the week','Aerobic exercise five times a week','Attends exercise classes','Anaerobic exercise 3+ times/wk', 'Competitive athlete',
        'FITT activity level 5; 12 or more occasions of vigorous activity in 4 weeks', 'Anaerobic exercise 2 times/wk', '30 minutes a day of at least moderate intensity physical activity on five or more days of the week']

In [199]:
# CATEGORISE THE STATUS OF PHYSICAL ACTIVITY PER PATIENT BASED ON MOST RECENT RECORD. 
def nearest(list, index):
    return min(list, key=lambda x: abs(x-index))

def PhysicalStat(ID,DATE):
    
    if ID in PHYSICAL_IDLIST:
        nearestdate=nearest(WLGP_PHYSICALACTIVITY.query('ALF_PE==@ID').EVENT_DT.tolist(),DATE)
        StatusDesc=WLGP_PHYSICALACTIVITY.query('ALF_PE==@ID & EVENT_DT==@nearestdate').EVENT_CD_DESC.tolist()
        if StatusDesc[0] in LIMITED:
            status=0
        elif StatusDesc[0] in REGULAR:
            status=1
    
    else:
        status='unknown'
    return status


In [200]:
# ADD THE PHYSICAL ACTIVITY VARIABLE TO THE DATASET 
DF_CHECK=WP2_df
DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE'], ignore_index=True)
DF_CHECK['PHYSICAL']=DF_CHECK.apply(lambda x: PhysicalStat(x['ALF_PE'],x['DATE']), axis=1)

WP2_df=pd.merge(WP2_df,DF_CHECK[['ALF_PE','DATE','PHYSICAL']])

In [203]:
#CHECK NUMBER OF PATIENTS THAT CHANGE CATEGORY
def PHYSChange(X):
    mylist=X.PHYSICAL.unique().tolist()
    if len(mylist)>1:
        physchange=1
    else:
        physchange=0
    return pd.Series(physchange, index=X.index)

WP2_df['PresencePHYSchange']=WP2_df.groupby('ALF_PE',group_keys=False).apply(PHYSChange)

INCLUDE COLUMNS TO CATEGORIZE PATIENTS' BMI

In [ ]:
WLGP_BMI.EVENT_CD_DESC.value_counts()

In [9]:
#DROP SOME EVENT_DESC NOT INDICATIVE OF A PATIENTS BMI
VALUES=['Body Mass Index']
WLGP_BMI=WLGP_BMI[~WLGP_BMI['EVENT_CD_DESC'].isin(VALUES)]

In [10]:
#SET DATES TO DATETIME TYPE
WLGP_BMI['EVENT_DT']=pd.to_datetime(WLGP_BMI['EVENT_DT'])

In [ ]:
#OBTAIN NUMBER OF UNIQUE IDS IN BMI RISK FACTOR TABLE
BMI_IDLIST=WLGP_BMI.ALF_PE.unique().tolist()
len(BMI_IDLIST)

In [ ]:
#NUMBER OF PATIENTS FROM OUR COHORT WHOSE IDs ARE IN BMI  RISK FACTORS TABLE
sum(list in WP2_df_IDLIST for list in BMI_IDLIST)

In [209]:
#MAKE LIST OF CATEGORIES
ObesityI=['Body mass index 30+ - obesity', 'Body Mass Index high K/M2']
ObesityIII=['Body mass index 40+ - severely obese']
OVERWEIGHT=['Body mass index index 25-29 - overweight']
NORMALWEIGHT=['Body mass index 20-24 - normal', 'Body Mass Index normal K/M2']
UNDERWEIGHT=['Body Mass Index low K/M2', 'Body mass index less than 20']

In [210]:
# CODE TO RECORD BMI STATUS BASED ON MOST RECENT RECORD
def BMIStat(ID,DATE):
    # OBTAINS THE BMI STATUS OF A PATIENT BY TAKING THE MOST RECENT RECORD FROM GP DATA SOURCE
    if ID in BMI_IDLIST:
        nearestdate=nearest(WLGP_BMI.query('ALF_PE==@ID').EVENT_DT.tolist(),DATE)
        StatusDesc=WLGP_BMI.query('ALF_PE==@ID & EVENT_DT==@nearestdate').EVENT_CD_DESC.tolist()
        if StatusDesc[0] in ObesityI:
            status='obesity class I'
        elif StatusDesc[0] in ObesityIII:
            status= 'obesity class III'
        elif StatusDesc[0] in OVERWEIGHT:
            status= 'pre-obesity'
        elif StatusDesc[0] in NORMALWEIGHT:
            status='normal weight'
        elif StatusDesc[0] in UNDERWEIGHT:
            status='underweight'  
    else:
        status='unknown'
    return status


In [211]:
# OBTAIN THE BMI VALUES PER PATIENTS AT UNIQUE ADMISSIONS AND ADD TO DATA
DF_CHECK=WP2_df
DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE'], ignore_index=True)
DF_CHECK['BMI']=DF_CHECK.apply(lambda x: BMIStat(x['ALF_PE'],x['DATE']), axis=1)

WP2_df=pd.merge(WP2_df,DF_CHECK[['ALF_PE','DATE','BMI']])

In [ ]:
WP2_df.drop_duplicates(subset=['ALF_PE','DATE'], ignore_index=True).BMI.value_counts()

In [215]:
#CHECK NUMBER OF PATIENTS THAT CHANGE CATEGORY
def BMIChange(X):
    mylist=X.BMI.unique().tolist()
    if len(mylist)>1:
        BMIchange=1
    else:
        BMIchange=0
    return pd.Series(BMIchange, index=X.index)

WP2_df['PresenceBMIchange']=WP2_df.groupby('ALF_PE',group_keys=False).apply(BMIChange)

In [ ]:
WP2_df.drop_duplicates(subset=['ALF_PE'])['PresenceBMIchange'].value_counts()

In [220]:
WP2_df[['ALF_PE', 'DATE', 'DISCH_DT', 'EPI_STR_DT', 'EPI_END_DT',
       'DIAG_NUM', 'DIAG_CODE', 'DIAG_DESC', 'PROV_UNIT_CD', 'CONDITION',
       'CODE_SYSTEM', 'WOB', 'AGE_AT_INDEX_DATE', 'AGE_GRPS_AT_INDEX_DATE',
       'AGE_GRPS_AT_INDEX_DATE_DESC', 'DOD', 'GNDR_CD', 'GNDR_DESC',
       'LSOA2011_CD_AT_INDEX_DATE', 'WIMD2019_QUINTILE_AT_INDEX_DATE',
       'WIMD2019_QUINTILE_AT_INDEX_DATE_DESC',
       'TOWNSEND2011_QUINTILE_AT_INDEX_DATE',
       'TOWNSEND2011_QUINTILE_AT_INDEX_DATE_DESC',
       'ETHN_EC_ONS_DATE_LATEST_CODE', 'ETHN_EC_ONS_DATE_LATEST_DESC',
       'ETHN_EC_NER_DATE_LATEST_CODE', 'ETHN_EC_NER_DATE_LATEST_DESC',
       'AGE_AT_DATE', 'AGEGRP_AT_DATE', 'MORTALITY', 'COM_SEQ',
       'COMORBIDITY_COUNT', 'AUTISM', 'BEHAVIOR_DISORDER', 'ALCOHOL_HISTORY',
       'SMOKING_HISTORY','MEDICATIONS','PHYSICAL', 'BMI']].to_csv('WP2_df_TOTALRISKFACTORS_01022024.csv')

COHORT ANALYSIS OF IN-PATIENT DATA

In [221]:
#DROPS DUPLICATES IN TOTAL DATA
WP2_df.drop_duplicates(inplace=True)

In [24]:
#IN-HOSPITAL DATA
WP2_df_PEDW=WP2_df.query('CODE_SYSTEM== "ICD10"')

#GP DATA
WP2_df_WLGP=WP2_df.query('CODE_SYSTEM== "READ"')

In [ ]:
# NUMBER OF PATIENTS WITH ADMISSIONS RECORDS
WP2_df_PEDW.ALF_PE.nunique()

In [ ]:
# OBTAIN RECORD OF ETHNIC GROUPS OF PATIENTS WITH ADMISSION RECORDS
WP2_df_PEDW.drop_duplicates(subset=['ALF_PE'], ignore_index=True).ETHN_EC_ONS_DATE_LATEST_DESC.value_counts()

In [ ]:
#OBTAIN THE NUMBER OF TIMES CONDITIONS WERE TREATED IN HOSPITAL & GP THROUGHOUT DATA
WP2_df.CONDITION.value_counts()

In [ ]:
#OBTAIN THE NUMBER OF TIMES CONDITIONS WERE TREATED IN HOSPITAL THROUGHOUT DATA
WP2_df_PEDW.CONDITION.value_counts()

DROP HOSPITAL DATA WITH MISSING EPISODE START DATE AND DISCHARGE DATE

In [ ]:
WP2_df_PEDW.DISCH_DT.isnull().sum().sum()

INPUT NEW COLUMN OF LENGTH OF STAY PER HOSPITAL ADMISSION

In [33]:
#INCLUDE NEW COLUMN OF LENGTH OF STAY FOR EACH HOSPITAL ADMISSION
WP2_df_PEDW['LOS']=(WP2_df_PEDW['DISCH_DT']-WP2_df_PEDW['DATE'])
# EXPRESS THE LOS AS timedelta64
WP2_df_PEDW.LOS=WP2_df_PEDW.LOS/np.timedelta64(1,'D')

In [ ]:
# TEST DATA FOR ADMISSIONS WITH NEGATIVE LOS, I.E., LOS<0
WP2_df_PEDW[(WP2_df_PEDW['LOS']<0)]

In [35]:
# DROP ROWS WITH NEGATIVE HOSPITAL STAYS 
WP2_df_PEDW.drop(WP2_df_PEDW[(WP2_df_PEDW['LOS']<0)].index, inplace=True)

OBTAIN COLUMN OF NUMBER OF ADMISSIONS IN THE PREVIOUS YEAR PER PATIENT

In [ ]:
DF_CHECK=WP2_df_PEDW
DF_CHECK=DF_CHECK.sort_values(by=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
s=DF_CHECK.groupby(['ALF_PE']).rolling('365D', on='DATE')['DATE'].count().sub(1).astype(int).to_frame('NUM_PRVADMISSION_1YR').reset_index()

WP2_df_PEDW.sort_values(by=['ALF_PE','DATE','DISCH_DT'], ignore_index=True)

DF_CHECK['NUM_PRVADMISSION_1YR']=s['NUM_PRVADMISSION_1YR']

WP2_df_PEDW=pd.merge(WP2_df_PEDW,DF_CHECK[['ALF_PE','DATE','DISCH_DT','NUM_PRVADMISSION_1YR']])
WP2_df_PEDW[['ALF_PE','DATE','DISCH_DT','NUM_PRVADMISSION_1YR']].head(40)

DROP DUPLICATE ROWS IN ENTIRE DATASET

In [241]:
#DROP DUPLICATES IN HOSPITAL DATA
WP2_df_PEDW.drop_duplicates(inplace=True)

In [ ]:
WP2_df_PEDW.shape[0]

In [253]:
WP2_df=WP2_df.reset_index(drop=True)
WP2_df_PEDW=WP2_df_PEDW.reset_index(drop=True)

OBTAIN COLUMN OF NUMBER OF HOSPITAL EPISODES IN THE PREVIOUS 1 YEAR PER PATIENT FROM ADMISSION DATE

In [ ]:
DF_CHECK=WP2_df_PEDW
DF_CHECK=DF_CHECK.sort_values(by=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT','EPI_STR_DT','EPI_END_DT','DIAG_CODE','DIAG_DESC'], ignore_index=True)
s=DF_CHECK.groupby(['ALF_PE']).rolling('365D', on='DATE')['DATE'].count().sub(1).astype(int).to_frame('NUM_PRVEPISODES_1YR').reset_index()
DF_CHECK['NUM_PRVEPISODES_1YR']=s['NUM_PRVEPISODES_1YR']

DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE','DISCH_DT'], ignore_index=True)
WP2_df_PEDW.sort_values(by=['ALF_PE','DATE','DISCH_DT'], ignore_index=True)

WP2_df_PEDW=pd.merge(WP2_df_PEDW,DF_CHECK[['ALF_PE','DATE','DISCH_DT','NUM_PRVEPISODES_1YR']])
WP2_df_PEDW[['ALF_PE','DATE','DISCH_DT','NUM_PRVEPISODES_1YR']]

OBTAIN COLUMN OF NUMBER OF COMORBIDITIES FROM HOSPITAL EPISODES IN THE PREVIOUS 1 YEAR PER PATIENT UP UNTIL ADMISSION DATE

In [255]:
WP2_df_PEDW['YEAR_BEFORE_ADMIS']= pd.to_datetime(WP2_df_PEDW.DATE)+pd.DateOffset(days=-365)
WP2_df_PEDW['3YEARS_BEFORE_ADMIS']= pd.to_datetime(WP2_df_PEDW.DATE)+pd.DateOffset(days=-1095)

In [256]:
def num1RYcomborbidity(x):
    #print(x.ALF_PE)
    ID= x['ALF_PE'].head(1).item()
    LB=x['YEAR_BEFORE_ADMIS'].head(1).item()
    UB=x['EPI_STR_DT'].head(1).item()
    Comorbid_counts=WP2_df_PEDW[(WP2_df_PEDW['ALF_PE']==ID ) & (WP2_df_PEDW['DATE']>= LB) &  (WP2_df_PEDW['DATE']<UB )]['CONDITION'].value_counts().shape[0]
    return pd.Series(Comorbid_counts, index=x.index)

In [257]:
WP2_df_PEDW['NUM_PRVCOMORBID_1YR']=WP2_df_PEDW.groupby(['ALF_PE','DATE', 'DISCH_DT'],group_keys=False).apply(num1RYcomborbidity)

OBTAIN COLUMN OF NUMBER OF COMORBIDITIES FROM HOSPITAL EPISODES IN THE PREVIOUS 3 YEARS PER PATIENT UP UNTIL ADMISSION DATE

In [258]:
def num3RYcomborbidity(x):
    #print(x.ALF_PE)
    ID= x['ALF_PE'].head(1).item()
    LB=x['3YEARS_BEFORE_ADMIS'].head(1).item()
    UB=x['EPI_STR_DT'].head(1).item()
    Comorbid_counts=WP2_df_PEDW[(WP2_df_PEDW['ALF_PE']==ID ) & (WP2_df_PEDW['DATE']>= LB) &  (WP2_df_PEDW['DATE']<UB )]['CONDITION'].value_counts().shape[0]
    return pd.Series(Comorbid_counts, index=x.index)

In [259]:
WP2_df_PEDW['NUM_PRVCOMORBID_3YR']=WP2_df_PEDW.groupby(['ALF_PE','DATE', 'DISCH_DT'],group_keys=False).apply(num3RYcomborbidity )

OBTAIN COLUMN OF NUMBER OF HOSPITAL EPISODES IN THE PREVIOUS 3 YEARS PER PATIENT FROM ADMISSION DATE

In [ ]:
DF_CHECK=WP2_df_PEDW
DF_CHECK=DF_CHECK.sort_values(by=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT','EPI_STR_DT','EPI_END_DT','DIAG_CODE','DIAG_DESC'], ignore_index=True)
s=DF_CHECK.groupby(['ALF_PE']).rolling('1095D', on='DATE')['DATE'].count().sub(1).astype(int).to_frame('NUM_PRVEPISODES_3YR').reset_index()
DF_CHECK['NUM_PRVEPISODES_3YR']=s['NUM_PRVEPISODES_3YR']

DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE','DISCH_DT'], ignore_index=True)
WP2_df_PEDW.sort_values(by=['ALF_PE','DATE','DISCH_DT'], ignore_index=True)

WP2_df_PEDW=pd.merge(WP2_df_PEDW,DF_CHECK[['ALF_PE','DATE','DISCH_DT','NUM_PRVEPISODES_3YR']])
WP2_df_PEDW[['ALF_PE','DATE','DISCH_DT','NUM_PRVEPISODES_3YR']]
#DF_CHECK

NUMBER OF HOSPITAL EPISODES WITHIN FIRST 24 HOURS OF ADMISSION 
ASSUMPTION: SINCE THE TIME STAMPS ARE NOT GIVEN FOR THE ADMISSION DATES, WE ASSUME THAT THE ADMISSION STARTED AT 00:00 HOURS FOR SIMPLICITY

In [261]:
WP2_df_PEDW['DATE_AFTER_ADMIS']= pd.to_datetime(WP2_df_PEDW.DATE)+pd.DateOffset(days=1)

In [263]:
def num24HRepisode(x):
    Episode_counts=x[x['EPI_STR_DT']<=x['DATE_AFTER_ADMIS']][['EPI_STR_DT','EPI_END_DT','DIAG_CODE']].value_counts().shape[0]
    return pd.Series(Episode_counts, index=x.index)

In [264]:
WP2_df_PEDW['NUMEPISODES_24HRS']=WP2_df_PEDW.groupby(['ALF_PE','DATE', 'DISCH_DT'],group_keys=False).apply(num24HRepisode)

NUMBER OF COMORBIDITIES WITHIN THE FIRST 24HOURS OF ADMISSION

In [265]:
def num24HRcomorbid(x):
    #OBTAINS THE TOTAL NUMBER OF COMORBIDITIES ASSOCIATED TO HOSPITAL EPISODES WITHIN 24 HOURS OF ADMISSION
    COMORBID_counts=x[x['EPI_STR_DT']<=x['DATE_AFTER_ADMIS']][['CONDITION']].value_counts().shape[0]
    return pd.Series(COMORBID_counts, index=x.index)

In [266]:
WP2_df_PEDW['NUMCOMORBIDITIES_24HRS']=WP2_df_PEDW.groupby(['ALF_PE','DATE', 'DISCH_DT'],group_keys=False).apply(num24HRcomorbid)

OBTAIN COLUMN OF NUMBER OF COMORBIDITIES PER ADMISSION

In [267]:
def numADMISSIONcomorbid(x):
    #OBTAINS THE TOTAL NUMBER OF COMORBIDITIES ASSOCIATED TO HOSPITAL EPISODES THROUGHOUT ADMISSION
    COMORBID_counts=x['CONDITION'].value_counts().shape[0]
    return pd.Series(COMORBID_counts, index=x.index)

In [268]:
WP2_df_PEDW['NUM_ADMISSION_COMORBIDITIES']=WP2_df_PEDW.groupby(['ALF_PE','DATE', 'DISCH_DT'],group_keys=False).apply(numADMISSIONcomorbid)

OBTAIN COLUMN OF NUMBER OF ADMISSIONS IN THE PREVIOUS 1 YEAR PER PATIENT PER CONDITION

In [ ]:
WP2_df_PEDW=WP2_df_PEDW.sort_values(by=['ALF_PE','CONDITION','DATE','DISCH_DT'], ignore_index=True)
s=WP2_df_PEDW.groupby(['ALF_PE','CONDITION']).rolling('365D', on='DATE')['DATE'].count().sub(1).astype(int).to_frame('NUM_PRVADMISSION_1YR_COND').reset_index()

s=s.sort_values(by=['ALF_PE','DATE'], ignore_index=True)
WP2_df_PEDW=WP2_df_PEDW.sort_values(by=['ALF_PE','DATE'], ignore_index=True)

WP2_df_PEDW['NUM_PRVADMISSION_1YR_COND']=s['NUM_PRVADMISSION_1YR_COND']
WP2_df_PEDW

OBTAIN COLUMN OF NUMBER OF ADMISSIONS IN THE PREVIOUS 3 YEARS PER PATIENT

In [ ]:
DF_CHECK=WP2_df_PEDW
DF_CHECK=DF_CHECK.sort_values(by=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
s=DF_CHECK.groupby(['ALF_PE']).rolling('1095D', on='DATE')['DATE'].count().sub(1).astype(int).to_frame('NUM_PRVADMISSION_3YR').reset_index()

WP2_df_PEDW.sort_values(by=['ALF_PE','DATE','DISCH_DT'], ignore_index=True)

s=s.sort_values(by=['ALF_PE','DATE'], ignore_index=True)
WP2_df_PEDW=WP2_df_PEDW.sort_values(by=['ALF_PE','DATE'], ignore_index=True)

DF_CHECK['NUM_PRVADMISSION_3YR']=s['NUM_PRVADMISSION_3YR']

WP2_df_PEDW=pd.merge(WP2_df_PEDW,DF_CHECK[['ALF_PE','DATE','DISCH_DT','NUM_PRVADMISSION_3YR']])
WP2_df_PEDW[['ALF_PE','DATE','DISCH_DT','NUM_PRVADMISSION_3YR']].head(40)

OBTAIN COLUMN OF NUMBER OF ADMISSIONS IN THE PREVIOUS 3 YEARS PER PATIENT PER CONDITION

In [271]:
WP2_df_PEDW=WP2_df_PEDW.sort_values(by=['ALF_PE','CONDITION','DATE','DISCH_DT'], ignore_index=True)
s=WP2_df_PEDW.groupby(['ALF_PE','CONDITION']).rolling('1095D', on='DATE')['DATE'].count().sub(1).astype(int).to_frame('NUM_PRVADMISSION_3YR_COND').reset_index()

s=s.sort_values(by=['ALF_PE','DATE'], ignore_index=True)
WP2_df_PEDW=WP2_df_PEDW.sort_values(by=['ALF_PE','DATE'], ignore_index=True)

WP2_df_PEDW['NUM_PRVADMISSION_3YR_COND']=s['NUM_PRVADMISSION_3YR_COND']
#WP2_df_PEDW

OBTAIN COLUMN OF TOTAL NUMBER OF HOSPITAL DAYS IN THE PREVIOUS 1 YEAR PER PATIENT

In [272]:
DF_CHECK=WP2_df_PEDW
DF_CHECK=DF_CHECK.sort_values(by=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
s=DF_CHECK.groupby(['ALF_PE']).rolling('365D', on='DATE')['LOS'].sum().astype(int).to_frame('NUM_PRVHOSPITAL_DAYS_1YR').reset_index()

s=s.sort_values(by=['ALF_PE','DATE'], ignore_index=True)
DF_CHECK=DF_CHECK.sort_values(by=['ALF_PE','DATE'], ignore_index=True)

DF_CHECK['NUM_PRVHOSPITAL_DAYS_1YR']=s['NUM_PRVHOSPITAL_DAYS_1YR']-DF_CHECK['LOS']

WP2_df_PEDW=pd.merge(WP2_df_PEDW,DF_CHECK[['ALF_PE','DATE','DISCH_DT','LOS','NUM_PRVHOSPITAL_DAYS_1YR']])

OBTAIN COLUMN OF TOTAL NUMBER OF HOSPITAL DAYS IN THE PREVIOUS 3 YEARS PER PATIENT

In [273]:
DF_CHECK=WP2_df_PEDW
DF_CHECK=DF_CHECK.sort_values(by=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
s=DF_CHECK.groupby(['ALF_PE']).rolling('1095D', on='DATE')['LOS'].sum().astype(int).to_frame('NUM_PRVHOSPITAL_DAYS_3YR').reset_index()

s=s.sort_values(by=['ALF_PE','DATE'], ignore_index=True)
DF_CHECK=DF_CHECK.sort_values(by=['ALF_PE','DATE'], ignore_index=True)

DF_CHECK['NUM_PRVHOSPITAL_DAYS_3YR']=s['NUM_PRVHOSPITAL_DAYS_3YR']-DF_CHECK['LOS']

WP2_df_PEDW=pd.merge(WP2_df_PEDW,DF_CHECK[['ALF_PE','DATE','DISCH_DT','LOS','NUM_PRVHOSPITAL_DAYS_3YR']])

OBTAIN COLUMN OF TOTAL NUMBER OF HOSPITALIZATION EVENTS (ADMISSIONS) AT END OF PATIENT RECORDS

In [274]:
DF_CHECK=WP2_df_PEDW
DF_CHECK=DF_CHECK.sort_values(by=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
DF_CHECK=DF_CHECK.drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)
s=DF_CHECK.groupby(['ALF_PE'])['DATE'].count().astype(int).to_frame('TOTAL_UNIQ_HOSPITALIZATIONS_AT_ENDDATE').reset_index()
WP2_df_PEDW=pd.merge(WP2_df_PEDW,s)

INCLUDE COLUMN FOR HOSPITAL MORTALITY

In [275]:
#INCLUDE COLUMN OF IN-HOSPITAL MORTALITY
def HOSP_M(x):
    if ((x['DATE']<=x['DOD']) and (x['DOD']<=x['DISCH_DT'])):
        hospm=1
    else:
        hospm=0 
    return hospm
    
WP2_df_PEDW['HOSP_MORTALITY']=WP2_df_PEDW.apply(lambda rows: HOSP_M(rows), axis=1)

In [276]:
#PREPROCESS COMORBIDITY COUNT: FOR MULTIPLE NEW DIAGNOSIS OCCURING ON THE SAME DAY (I.E., EPISODE DATE), THE COMORBIDITY COUNT SHOULD BE THE SAME, AND NOT AN INCREASING SEQUENCE. 
WP2_df_PEDW['COMORBIDITY_COUNT']=WP2_df_PEDW.groupby(['ALF_PE','DATE', 'DISCH_DT', 'EPI_STR_DT'],group_keys=False)['COMORBIDITY_COUNT'].transform(lambda x: max(x))

In [278]:
# SAVE THE PREPROCESSED DATA 
WP2_df_PEDW.to_csv('WP2_df_PEDW_TOTALEPISODES.csv')

### ANALYSIS SECTION OF DATA

#### BOTH GP AND HOSPITAL

##### OBTAIN DISTRIBUTION OF MEAN COMORBIDITY COUNTS ACROSS AGEGROUP, ETHNICITY AND GENDER

In [ ]:
datachk=WP2_df.drop_duplicates(subset=['ALF_PE','DATE'], keep='last')
meancomorb=datachk.groupby(['AGEGRP_AT_DATE'])['COMORBIDITY_COUNT'].agg(['count','mean', 'std']).reset_index()
meancomorb.to_csv('MEANSTD_COMMORBIDITIES_AGEGROUP_GP_PEDW.csv')
#sns.barplot(data=meancomorb, x='AGEGRP_AT_ADMIS_DT', y='std',  ci=False)
fig = plt.figure()
pp=meancomorb.plot(x='AGEGRP_AT_DATE',y=['mean','std'], kind='bar',color=[(0.53,0.81,0.92),(0.95,0.55,0)])
pp.set_xticklabels(pp.get_xticklabels(), rotation=0)
plt.xlabel('AGE GROUP')
plt.ylabel('MEAN COMORBIDITY COUNT')
plt.legend()

plt.savefig('MEANSTD_COMMORBIDITIES_AGEGROUP_GP_PEDW.png', dpi=300, bbox_inches= 'tight',pad_inches=0.0)
plt.show(block=False)

In [ ]:
width=0.1
fig = plt.figure()
Group=['<20','20-29','30-39','40-49','50-59','60-69','70-79','80+']
X_axis= np.arange(len(Group))

plt.bar(X_axis, meancomorb['count'].tolist(), width)
plt.xticks(X_axis, Group)
# meancomorb.plot(x='AGEGRP_AT_DATE',y='count', kind='bar')
plt.ylabel('Counts')
plt.xlabel('Age Groups')
#plt.title('Mean Number of Comorbidities Across Gender and Age Groups')
plt.show(block=False)
fig.savefig("AGEGROUPDIST_WP2DF.png", dpi=300, bbox_inches= "tight")
# plt.yscale('log')




In [ ]:
datachk=WP2_df.drop_duplicates(subset=['ALF_PE','DATE'], keep='last')
meancomorb=datachk.groupby(['AGEGRP_AT_DATE','GNDR_DESC'])['COMORBIDITY_COUNT'].agg(['count','mean', 'std']).reset_index()
meancomorb.to_csv('MEANSTD_COMMORBIDITIES_AGEGROUP_GP_PEDW_GENDER.csv')
fig = plt.figure()
plt.style.use('seaborn-whitegrid')
MALES=meancomorb.query('GNDR_DESC=="MALE"')['mean'].tolist()
FEMALES=meancomorb.query('GNDR_DESC=="FEMALE"')['mean'].tolist()
Group=['<20','20-29','30-39','40-49','50-59','60-69','70-79','80+']
X_axis= np.arange(len(Group))
bars=plt.bar(X_axis-0.2, MALES, 0.4, label='MALE', color=(0.53,0.81,0.92))#(0.65,0.8,1)
# bars[1].set_color('orange')
bars2=plt.bar(X_axis+0.2, FEMALES, 0.4, label='FEMALE', color=(0.95,0.55,0))#1,0.456,0
# bars2[1].set_color('black')

plt.xticks(X_axis, Group)
# plt.set_xticklabels(pp.get_xticklabels(), rotation=0)
plt.xlabel('AGE GROUP')
plt.ylabel('MEAN COMORBIDITY COUNT')
# plt.title('Mean Number of Comorbidities Across Gender and Age Groups')
plt.legend()
plt.show()
fig.savefig("MEANSTD_COMMORBIDITIES_AGEGROUP_GP_PEDW_GENDER.png", dpi=300, bbox_inches= "tight")

In [ ]:
plt.bar(X_axis-0.2, meancomorb.query('GNDR_DESC=="MALE"')['count'].tolist(), 0.4, label='MALE')
plt.bar(X_axis+0.2, meancomorb.query('GNDR_DESC=="FEMALE"')['count'].tolist(), 0.4, label='FEMALE')
plt.xticks(X_axis, Group)
plt.legend()
plt.show()

In [ ]:
datachk=WP2_df.drop_duplicates(subset=['ALF_PE','DATE'], keep='last')
meancomorb=datachk.groupby(['AGEGRP_AT_DATE','ETHN_EC_ONS_DATE_LATEST_DESC'])['COMORBIDITY_COUNT'].agg(['count','mean', 'std']).reset_index()
meancomorb.to_csv('MEANSTD_COMMORBIDITIES_AGEGROUP_GP_PEDW_ETHNICITY.csv')
#meancomorb
fig = plt.figure(figsize=(20,7))
ASIAN=meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Asian"')['mean'].tolist()
BLACK=meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Black"')['mean'].tolist()
MIXED=meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Mixed"')['mean'].tolist()
OTHER=meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Other"')['mean'].tolist()
UNKNOWN=meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Unknown"')['mean'].tolist()
WHITE=meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="White"')['mean'].tolist()

Group=['<20','20-29','30-39','40-49','50-59','60-69','70-79','80+']
X_axis= np.arange(len(Group))
width=0.1
plt.bar(X_axis-width*3, ASIAN, width, label='ASIAN')
plt.bar(X_axis-width*2, BLACK, width, label='BLACK')
plt.bar(X_axis-width, MIXED, width, label='MIXED')
plt.bar(X_axis, OTHER, width, label='OTHER')
plt.bar(X_axis+width, UNKNOWN, width, label='UNKNOWN')
plt.bar(X_axis+width*2, WHITE, width, label='WHITE')

plt.xticks(X_axis, Group)
plt.xlabel('Age Groups')
plt.ylabel('Mean Number of Comorbidities')
#plt.title('Mean Number of Comorbidities Across Gender and Age Groups')
plt.legend()
plt.show()
fig.savefig("MEANSTD_COMMORBIDITIES_AGEGROUP_GP_PEDW_ETHNICITY.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
width=0.1
plt.bar(X_axis-width*3, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Asian"')['count'].tolist(), width, label='ASIAN')
plt.bar(X_axis-width*2, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Black"')['count'].tolist(), width, label='BLACK')
plt.bar(X_axis-width, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Mixed"')['count'].tolist(), width, label='MIXED')
plt.bar(X_axis, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Other"')['count'].tolist(), width, label='OTHER')
plt.bar(X_axis+width, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Unknown"')['count'].tolist(), width, label='UNKNOWN')
plt.bar(X_axis+width*2, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="White"')['count'].tolist(), width, label='WHITE')

plt.xticks(X_axis, Group)
plt.ylabel('Counts')
plt.xlabel('Age Groups')
#plt.title('Mean Number of Comorbidities Across Gender and Age Groups')
plt.legend()
fig.savefig("AGEGROUPDIST_WP2DF_ETHNICITY.png", dpi=300, bbox_inches= "tight")
# plt.yscale('log')


plt.show(block=False)

In [ ]:
#fig = plt.figure(figsize=(20,7))
width=0.1
plt.bar(X_axis-width*3, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Asian"')['count'].tolist(), width, label='ASIAN')
plt.bar(X_axis-width*2, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Black"')['count'].tolist(), width, label='BLACK')
plt.bar(X_axis-width, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Mixed"')['count'].tolist(), width, label='MIXED')
plt.bar(X_axis, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Other"')['count'].tolist(), width, label='OTHER')
plt.bar(X_axis+width, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Unknown"')['count'].tolist(), width, label='UNKNOWN')
plt.bar(X_axis+width*2, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="White"')['count'].tolist(), width, label='WHITE')

plt.ylabel('Counts')
plt.xlabel('Age Groups')
plt.yscale('log')
plt.xticks(X_axis, Group)
plt.legend()
plt.show()

fig.savefig("AGEGROUPDIST_WP2DF_ETHNICITY_log.png", dpi=300, bbox_inches= "tight")

#### ONLY HOSPITAL DATA

##### Demographic Cohort Description

##### OBTAIN DISTRIBUTION OF MEAN COMORBIDITY COUNTS ACROSS AGEGROUP, ETHNICITY AND GENDER

In [ ]:
WP2_df_PEDW['DATE']=pd.to_datetime(df['DATE'])

In [ ]:
fig = plt.figure()
Group=['<20','20-29','30-39','40-49','50-59','60-69','70-79','80+']
X_axis= np.arange(len(Group))

plt.bar(X_axis, meancomorb['count'].tolist(), width, label='ASIAN')
plt.xticks(X_axis, Group)
# meancomorb.plot(x='AGEGRP_AT_DATE',y='count', kind='bar')
plt.ylabel('Counts')
plt.xlabel('Age Groups')
#plt.title('Mean Number of Comorbidities Across Gender and Age Groups')
plt.show(block=False)
fig.savefig("AGEGROUPDIST_PEDW.png", dpi=300, bbox_inches= "tight")
# plt.yscale('log')




In [ ]:
plt.bar(X_axis-0.2, meancomorb.query('GNDR_DESC=="MALE"')['count'].tolist(), 0.4, label='MALE')
plt.bar(X_axis+0.2, meancomorb.query('GNDR_DESC=="FEMALE"')['count'].tolist(), 0.4, label='FEMALE')
plt.xticks(X_axis, Group)
plt.legend()
plt.show()

In [ ]:
datachk=WP2_df_PEDW.drop_duplicates(subset=['ALF_PE','DATE'], keep='last')
meancomorb=datachk.groupby(['AGEGRP_AT_DATE','ETHN_EC_ONS_DATE_LATEST_DESC'])['COMORBIDITY_COUNT'].agg(['count','mean', 'std']).reset_index()
meancomorb.to_csv('MEANSTD_COMMORBIDITIES_AGEGROUP_PEDW_ETHNICITY.csv')
#meancomorb
fig = plt.figure(figsize=(20,7))
ASIAN=meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Asian"')['mean'].tolist()
BLACK=meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Black"')['mean'].tolist()
MIXED=meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Mixed"')['mean'].tolist()
OTHER=meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Other"')['mean'].tolist()
UNKNOWN=meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Unknown"')['mean'].tolist()
WHITE=meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="White"')['mean'].tolist()

Group=['<20','20-29','30-39','40-49','50-59','60-69','70-79','80+']
X_axis= np.arange(len(Group))
width=0.1
plt.bar(X_axis-width*3, ASIAN, width, label='ASIAN')
plt.bar(X_axis-width*2, BLACK, width, label='BLACK')
plt.bar(X_axis-width, MIXED, width, label='MIXED')
plt.bar(X_axis, OTHER, width, label='OTHER')
plt.bar(X_axis+width, UNKNOWN, width, label='UNKNOWN')
plt.bar(X_axis+width*2, WHITE, width, label='WHITE')

plt.xticks(X_axis, Group)
plt.xlabel('Age Groups')
plt.ylabel('Mean Number of Comorbidities')
#plt.title('Mean Number of Comorbidities Across Gender and Age Groups')
plt.legend()
plt.show()
fig.savefig("MEANSTD_COMMORBIDITIES_AGEGROUP_PEDW_ETHNICITY.png", dpi=300, bbox_inches= "tight")

In [ ]:
fig = plt.figure()
width=0.1
plt.bar(X_axis-width*3, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Asian"')['count'].tolist(), width, label='ASIAN')
plt.bar(X_axis-width*2, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Black"')['count'].tolist(), width, label='BLACK')
plt.bar(X_axis-width, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Mixed"')['count'].tolist(), width, label='MIXED')
plt.bar(X_axis, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Other"')['count'].tolist(), width, label='OTHER')
plt.bar(X_axis+width, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Unknown"')['count'].tolist(), width, label='UNKNOWN')
plt.bar(X_axis+width*2, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="White"')['count'].tolist(), width, label='WHITE')

plt.xticks(X_axis, Group)
plt.ylabel('Counts')
plt.xlabel('Age Groups')
#plt.title('Mean Number of Comorbidities Across Gender and Age Groups')
plt.legend()
fig.savefig("AGEGROUPDIST_PEDW_ETHNICITY.png", dpi=300, bbox_inches= "tight")
# plt.yscale('log')


plt.show(block=False)

In [ ]:
#fig = plt.figure(figsize=(20,7))
width=0.1
plt.bar(X_axis-width*3, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Asian"')['count'].tolist(), width, label='ASIAN')
plt.bar(X_axis-width*2, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Black"')['count'].tolist(), width, label='BLACK')
plt.bar(X_axis-width, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Mixed"')['count'].tolist(), width, label='MIXED')
plt.bar(X_axis, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Other"')['count'].tolist(), width, label='OTHER')
plt.bar(X_axis+width, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="Unknown"')['count'].tolist(), width, label='UNKNOWN')
plt.bar(X_axis+width*2, meancomorb.query('ETHN_EC_ONS_DATE_LATEST_DESC=="White"')['count'].tolist(), width, label='WHITE')

plt.ylabel('Counts')
plt.xlabel('Age Groups')
plt.yscale('log')
plt.xticks(X_axis, Group)
plt.legend()
plt.show()

fig.savefig("AGEGROUPDIST_PEDW_ETHNICITY_log.png", dpi=300, bbox_inches= "tight")

END

In [354]:
#FILTER DATA TO DROP DUPLICATE ADMISSIONS PER CONDITION FOR EACH PATIENT
WP2_df_PEDW=WP2_df_PEDW.drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT','CONDITION'], ignore_index=True)

OBTAIN A THESHOLD FOR BINARY CLASSIFICATION OF LOS

In [355]:
#WE OBTAIN THE STATISTICS WITH RESPECT TO UNIQUE ADMISSIONS, HENCE, WE DROP DUPLICATE ADMISSIONS PER PATIENTS
PEDW_UNIQ=WP2_df_PEDW.drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True)

In [356]:
#REPLACE MISSING WIMD DATA WITH 'unknown'
PEDW_UNIQ['WIMD2019_QUINTILE_AT_INDEX_DATE'].fillna("Unknown", inplace=True)

In [ ]:
#INTERQUARTILE RANGE
IQR= PEDW_UNIQ.LOS.quantile(0.75)-PEDW_UNIQ.LOS.quantile(0.25)

#UPPER BOUNDARY
UPPER_BOUND=PEDW_UNIQ.LOS.quantile(0.75) + (1.5*IQR)
UPPER_BOUND

In [ ]:
#OBTAIN MEAN LOS WITHOUT OUTLIERS
LOS_THRESH=PEDW_UNIQ.query('LOS<=@UPPER_BOUND').LOS.mean()
LOS_THRESH

In [ ]:
import math 
LOS_THRESH=math.ceil(LOS_THRESH)
LOS_THRESH

In [ ]:
#DESCRIPTION OF DATA WITHOUT OUTLIERS
PEDW_UNIQ.query('LOS<=@UPPER_BOUND').LOS.describe()

INCLUDE COLUMN FOR BINARY CLASSIFICATION OF LOS

In [12]:
WP2_df_PEDW['LOSClass']=0
WP2_df_PEDW.loc[WP2_df_PEDW['LOS']>=LOS_THRESH, 'LOSClass']=1

In [ ]:
WP2_df_PEDW.LOSClass.value_counts()

In [400]:
WP2_df_PEDW.columns

Index(['ALF_PE', 'DATE', 'DISCH_DT', 'EPI_STR_DT', 'EPI_END_DT', 'DIAG_NUM',
       'DIAG_CODE', 'DIAG_DESC', 'PROV_UNIT_CD', 'CONDITION', 'CODE_SYSTEM',
       'WOB', 'AGE_AT_INDEX_DATE', 'AGE_GRPS_AT_INDEX_DATE',
       'AGE_GRPS_AT_INDEX_DATE_DESC', 'DOD', 'GNDR_CD', 'GNDR_DESC',
       'LSOA2011_CD_AT_INDEX_DATE', 'WIMD2019_QUINTILE_AT_INDEX_DATE',
       'WIMD2019_QUINTILE_AT_INDEX_DATE_DESC',
       'TOWNSEND2011_QUINTILE_AT_INDEX_DATE',
       'TOWNSEND2011_QUINTILE_AT_INDEX_DATE_DESC',
       'ETHN_EC_ONS_DATE_LATEST_CODE', 'ETHN_EC_ONS_DATE_LATEST_DESC',
       'ETHN_EC_NER_DATE_LATEST_CODE', 'ETHN_EC_NER_DATE_LATEST_DESC',
       'AGE_AT_DATE', 'AGEGRP_AT_DATE', 'MORTALITY', 'COM_SEQ',
       'COMORBIDITY_COUNT', 'AUTISM', 'BEHAVIOR_DISORDER', 'ALCOHOL_HISTORY',
       'PresenceAlcRec', 'PresenceALCchange', 'SMOKING_HISTORY',
       'PresenceSMOKchange', 'PresenceSmokRec', 'MEDICATIONS',
       'PresenceMEDIchange', 'PHYSICAL', 'PresencePHYSRec',
       'PresencePHYSchange', 

ANALYZING THE OUTLIERS

In [368]:
WP2_df_PEDW_OUTLIERS1=PEDW_UNIQ.query('LOS>=@UPPER_BOUND')

In [ ]:
# Obtain counts of admissions with LOS above upperbound
WP2_df_PEDW_OUTLIERS1.shape[0]

In [ ]:
LOS_box1=WP2_df_PEDW_OUTLIERS1.boxplot(column='LOS')
LOS_box1.plot()
plt.yscale('log')
plt.show()

In [ ]:
WP2_df_PEDW_OUTLIERS1.LOS.describe()

In [372]:
#INTERQUARTILE RANGE FOR OUTLIERS
IQR1= WP2_df_PEDW_OUTLIERS1.LOS.quantile(0.75)-WP2_df_PEDW_OUTLIERS1.LOS.quantile(0.25)

#UPPER BOUNDARY FOR BOX PLOT OF OUTLIERS
UPPER_BOUND1=WP2_df_PEDW_OUTLIERS1.LOS.quantile(0.75) + (1.5*IQR1)

#REMARK: THE OUTLIERS ARE PATIENTS WITH LOS>UPPER_BOUND

In [ ]:
WP2_df_PEDW_OUTLIERS1.query('LOS<=@UPPER_BOUND1').LOS.mean()

In [375]:
WP2_df_PEDW_OUTLIERS2=WP2_df_PEDW_OUTLIERS1.query('LOS>@UPPER_BOUND1')

In [ ]:
LOS_box2=WP2_df_PEDW_OUTLIERS2.boxplot(column='LOS')
LOS_box2.plot()
plt.yscale('log')
plt.show()

In [ ]:
WP2_df_PEDW_OUTLIERS2.LOS.describe()

In [ ]:
#INTERQUARTILE RANGE FOR OUTLIERS
IQR2= WP2_df_PEDW_OUTLIERS2.LOS.quantile(0.75)-WP2_df_PEDW_OUTLIERS2.LOS.quantile(0.25)

#UPPER BOUNDARY FOR BOX PLOT OF OUTLIERS
UPPER_BOUND2=WP2_df_PEDW_OUTLIERS2.LOS.quantile(0.75) + (1.5*IQR2)
UPPER_BOUND2
#REMARK: THE OUTLIERS ARE PATIENTS WITH LOS>UPPER_BOUND

In [ ]:
WP2_df_PEDW_OUTLIERS2.query('LOS<=@UPPER_BOUND2').LOS.mean()

In [ ]:
#TOTAL NUMBER OF UNIQUE ADMISSIONS ABOVE UPPER_BOUND1
Numberadm=WP2_df_PEDW.query('LOS>=@UPPER_BOUND1').drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT'], ignore_index=True).shape[0]
Numberadm

In [ ]:
WP2_df_PEDW.query('LOS>=@UPPER_BOUND1')

In [382]:
#OBTAIN COUNTS OF CONDITIONS FOR LOS> UPPER_BOUND1
((WP2_df_PEDW.query('LOS>=@UPPER_BOUND1').CONDITION.value_counts()/Numberadm)*100).to_csv('CONDITIONS_LOS_ABOVE_UB1.csv')

In [ ]:
# Display conditions of patients with LOS>UPPER_BOUND1 days
outlied=(WP2_df_PEDW.query('LOS>=@UPPER_BOUND1'))
fig=plt.figure(figsize=(7,23))
sns.countplot(y="CONDITION", data=outlied)
plt.title('Conditions of patients with LOS>129 Days')
plt.show()
fig.savefig("OUTLIERCOND_count.png", dpi=300, bbox_inches= "tight")

ANALYZE MENTAL ILLNES PATIENTS ABOVE UPPERBOUND1 (OUTLIERS)

In [ ]:
# plots of patient ages for LOS>129 
WP2_df_PEDW.query('LOS>=@UPPER_BOUND1 and CONDITION=="MENTALILLNESS"').AGE_AT_ADMIS_DT.hist()
plt.xlabel("AGE")
plt.ylabel("COUNTS")
plt.show()

In [ ]:
# plots of patient Agegroups for LOS>129
(WP2_df_PEDW.query('LOS>=@UPPER_BOUND1 and CONDITION=="MENTALILLNESS"').AGEGRP_AT_ADMIS_DT.value_counts(normalize=True)*100).plot.bar()
plt.xlabel("AGE GROUP")
plt.ylabel("COUNTS")
plt.show()

In [ ]:
out_data=WP2_df_PEDW.query('LOS>129 and CONDITION=="MENTALILLNESS"')
out_data.hist(column='LOS',bins=10)
#plt.xlabel("LOS")
plt.ylabel("COUNTS")
plt.show()

In [ ]:
#OBTAIN COUNTS OF COMORBIDITIES PER PATIENTS AT EACH UNIQUE ADMISSIONS
WP2_df_PEDW.query('LOS>=@UPPER_BOUND1').drop_duplicates(subset=['ALF_PE','DATE', 'DISCH_DT'],keep='last', ignore_index=True).COMORBIDITY_COUNT.value_counts(normalize=True)

In [ ]:
# PLOT COUNTS OF COMORBIDITIES PER PATIENTS AT EACH UNIQUE ADMISSIONS
WP2_df_PEDW.query('LOS>=@UPPER_BOUND1 and CONDITION=="MENTALILLNESS"').COMORBIDITY_COUNT.value_counts(normalize=True).plot.bar()
plt.xlabel("COMORBIDITY COUNTS")
plt.ylabel("COUNTS")
plt.show()

In [ ]:
WP2_df_PEDW.query('LOS>=@UPPER_BOUND1 and CONDITION=="MENTALILLNESS"').ETHN_EC_ONS_DATE_LATEST_DESC.value_counts(normalize=True)

In [395]:
WP2_df_PEDW.query('LOS>=@UPPER_BOUND1 and CONDITION=="MENTALILLNESS"').AGEGRP_AT_ADMIS_DT.value_counts(normalize=False)

In [ ]:
WP2_df_PEDW.query('LOS>=@UPPER_BOUND1').drop_duplicates(subset=['ALF_PE'], ignore_index=True).GNDR_DESC.value_counts()

In [ ]:
PEDW_UNIQ.query('LOS>=@UPPER_BOUND1').COMORBIDITY_COUNT.value_counts(normalize=False)

# ANALYSIS OF FIRST QUANTILE 25%-50% (0-2 days)

## UNGROUPED

In [ ]:
#NUMBER OF UNIQUE PATIENTS
PEDW_UNIQ.query('LOS<=2').drop_duplicates(subset=['ALF_PE'], ignore_index=True).shape[0]

In [ ]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS<=2').shape[0]

In [ ]:
# obtain distribution of comorbidity counts per patients across all unique admissions with LOS<=2
PEDW_UNIQ.query('LOS<=2').NUM_ADMISSION_COMORBIDITIES.describe()

In [ ]:
PEDW_UNIQ.query('LOS<=2').LOS.describe()

## ETHNICITY

In [390]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS<=2').drop_duplicates(subset=['ALF_PE'], ignore_index=True).ETHN_EC_ONS_DATE_LATEST_DESC.value_counts().to_csv("25_50ethn_patientcount.csv")

In [391]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS<=2').ETHN_EC_ONS_DATE_LATEST_DESC.value_counts().to_csv("25_50ethn_count.csv")

In [392]:
PEDW_UNIQ.query('LOS<=2').groupby(['ETHN_EC_ONS_DATE_LATEST_DESC'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("25_50Numcom_ethn.csv")

In [393]:
PEDW_UNIQ.query('LOS<=2').groupby(['ETHN_EC_ONS_DATE_LATEST_DESC'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("25_50LOSethn.csv")

## GENDER

In [394]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS<=2').drop_duplicates(subset=['ALF_PE'], ignore_index=True).GNDR_DESC.value_counts().to_csv("25_50gndr_patientcount.csv")

In [395]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS<=2').GNDR_DESC.value_counts().to_csv("25_50GNDR_count.csv")

In [396]:
PEDW_UNIQ.query('LOS<=2').groupby(['GNDR_DESC'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("25_50Numcom_gndr.csv")

In [397]:
PEDW_UNIQ.query('LOS<=2').groupby(['GNDR_DESC'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("25_50LOSgndr.csv")

## AGE GROUP

In [398]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS<=2').drop_duplicates(subset=['ALF_PE'], ignore_index=True).AGEGRP_AT_ADMIS_DT.value_counts().to_csv("25_50agegrp_patientcount.csv")

In [399]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS<=2').AGEGRP_AT_ADMIS_DT.value_counts().to_csv("25_50agegrp_count.csv")

In [400]:
PEDW_UNIQ.query('LOS<=2').groupby(['AGEGRP_AT_ADMIS_DT'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("25_50Numcom_agegrp.csv")

In [401]:
PEDW_UNIQ.query('LOS<=2').groupby(['AGEGRP_AT_ADMIS_DT'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("25_50LOSagegrp.csv")

## WIMD

In [412]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS<=2').drop_duplicates(subset=['ALF_PE'], ignore_index=True).WIMD2019_QUINTILE_AT_INDEX_DATE.value_counts().to_csv("25_50wimd_patientcount.csv")

In [413]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS<=2').WIMD2019_QUINTILE_AT_INDEX_DATE.value_counts().to_csv("25_50wimd_count.csv")

In [415]:
PEDW_UNIQ.query('LOS<=2').groupby(['WIMD2019_QUINTILE_AT_INDEX_DATE'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("25_50Numcom_wimd.csv")

In [416]:
PEDW_UNIQ.query('LOS<=2').groupby(['WIMD2019_QUINTILE_AT_INDEX_DATE'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("25_50LOSwimd.csv")

# ANALYSIS OF QUANTILE 50%-75% (2-8 days)

## UNGROUPED

In [ ]:
#NUMBER OF UNIQUE PATIENTS
PEDW_UNIQ.query('LOS>=2 and LOS<=8').drop_duplicates(subset=['ALF_PE'], ignore_index=True).shape[0]

In [ ]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>=2 and LOS<=8').shape[0]

In [ ]:
# obtain distribution of comorbidity counts per patients across all unique admissions with LOS>=2 and LOS<=8
PEDW_UNIQ.query('LOS>=2 and LOS<=8').NUM_ADMISSION_COMORBIDITIES.describe()

## ETHNICITY

In [421]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS>=2 and LOS<=8').drop_duplicates(subset=['ALF_PE'], ignore_index=True).ETHN_EC_ONS_DATE_LATEST_DESC.value_counts().to_csv("50_75ethn_patientcount.csv")

In [422]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>=2 and LOS<=8').ETHN_EC_ONS_DATE_LATEST_DESC.value_counts().to_csv("50_75ethn_count.csv")

In [423]:
PEDW_UNIQ.query('LOS>=2 and LOS<=8').groupby(['ETHN_EC_ONS_DATE_LATEST_DESC'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("50_75Numcom_ethn.csv")

In [424]:
PEDW_UNIQ.query('LOS>=2 and LOS<=8').groupby(['ETHN_EC_ONS_DATE_LATEST_DESC'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("50_75LOSethn.csv")

## GENDER

In [425]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS>=2 and LOS<=8').drop_duplicates(subset=['ALF_PE'], ignore_index=True).GNDR_DESC.value_counts().to_csv("50_75gndr_patientcount.csv")

In [426]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>=2 and LOS<=8').GNDR_DESC.value_counts().to_csv("50_75GNDR_count.csv")

In [427]:
PEDW_UNIQ.query('LOS<=2').groupby(['GNDR_DESC'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("50_75Numcom_gndr.csv")

In [428]:
PEDW_UNIQ.query('LOS>=2 and LOS<=8').groupby(['GNDR_DESC'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("50_75LOSgndr.csv")

## AGE GROUP

In [429]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS>=2 and LOS<=8').drop_duplicates(subset=['ALF_PE'], ignore_index=True).AGEGRP_AT_ADMIS_DT.value_counts().to_csv("50_75agegrp_patientcount.csv")

In [430]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>=2 and LOS<=8').AGEGRP_AT_ADMIS_DT.value_counts().to_csv("50_75agegrp_count.csv")

In [431]:
PEDW_UNIQ.query('LOS>=2 and LOS<=8').groupby(['AGEGRP_AT_ADMIS_DT'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("50_75Numcom_agegrp.csv")

In [432]:
PEDW_UNIQ.query('LOS>=2 and LOS<=8').groupby(['AGEGRP_AT_ADMIS_DT'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("50_75LOSagegrp.csv")

## WIMD

In [433]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS>=2 and LOS<=8').drop_duplicates(subset=['ALF_PE'], ignore_index=True).WIMD2019_QUINTILE_AT_INDEX_DATE.value_counts().to_csv("50_75wimd_patientcount.csv")

In [434]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>=2 and LOS<=8').WIMD2019_QUINTILE_AT_INDEX_DATE.value_counts().to_csv("50_75wimd_count.csv")

In [435]:
PEDW_UNIQ.query('LOS>=2 and LOS<=8').groupby(['WIMD2019_QUINTILE_AT_INDEX_DATE'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("50_75Numcom_wimd.csv")

In [436]:
PEDW_UNIQ.query('LOS>=2 and LOS<=8').groupby(['WIMD2019_QUINTILE_AT_INDEX_DATE'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("50_75LOSwimd.csv")

# ANALYSIS OF QUANTILE 75%-UPPERBOUND (8-20 days)

## UNGROUPED

In [ ]:
#NUMBER OF UNIQUE PATIENTS
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').drop_duplicates(subset=['ALF_PE'], ignore_index=True).shape[0]

In [ ]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').shape[0]

In [ ]:
# obtain distribution of comorbidity counts per patients across all unique admissions with LOS>=8 and LOS<=@UPPER_BOUND
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').NUM_ADMISSION_COMORBIDITIES.describe()

## ETHNICITY

In [446]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').drop_duplicates(subset=['ALF_PE'], ignore_index=True).ETHN_EC_ONS_DATE_LATEST_DESC.value_counts().to_csv("75_UBethn_patientcount.csv")

In [447]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').ETHN_EC_ONS_DATE_LATEST_DESC.value_counts().to_csv("75_UBethn_count.csv")

In [448]:
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').groupby(['ETHN_EC_ONS_DATE_LATEST_DESC'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("75_UBNumcom_ethn.csv")

In [449]:
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').groupby(['ETHN_EC_ONS_DATE_LATEST_DESC'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("75_UBLOSethn.csv")

## GENDER

In [442]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').drop_duplicates(subset=['ALF_PE'], ignore_index=True).GNDR_DESC.value_counts().to_csv("75_UBgndr_patientcount.csv")

In [443]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').GNDR_DESC.value_counts().to_csv("75_UBGNDR_count.csv")

In [444]:
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').groupby(['GNDR_DESC'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("75_UBNumcom_gndr.csv")

In [445]:
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').groupby(['GNDR_DESC'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("75_UBLOSgndr.csv")

## AGE GROUP

In [450]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').drop_duplicates(subset=['ALF_PE'], ignore_index=True).AGEGRP_AT_ADMIS_DT.value_counts().to_csv("75_UBagegrp_patientcount.csv")

In [451]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').AGEGRP_AT_ADMIS_DT.value_counts().to_csv("75_UBagegrp_count.csv")

In [452]:
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').groupby(['AGEGRP_AT_ADMIS_DT'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("75_UBNumcom_agegrp.csv")

In [453]:
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').groupby(['AGEGRP_AT_ADMIS_DT'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("75_UBLOSagegrp.csv")

## WIMD

In [458]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').drop_duplicates(subset=['ALF_PE'], ignore_index=True).WIMD2019_QUINTILE_AT_INDEX_DATE.value_counts().to_csv("75_UBwimd_patientcount.csv")

In [459]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').WIMD2019_QUINTILE_AT_INDEX_DATE.value_counts().to_csv("75_UBwimd_count.csv")

In [460]:
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').groupby(['WIMD2019_QUINTILE_AT_INDEX_DATE'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("75_UBNumcom_wimd.csv")

In [461]:
PEDW_UNIQ.query('LOS>=8 and LOS<=@UPPER_BOUND').groupby(['WIMD2019_QUINTILE_AT_INDEX_DATE'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("75_UBLOSwimd.csv")

# ANALYSIS OF STAYS >UPPERBOUND

## UNGROUPED

In [ ]:
#NUMBER OF UNIQUE PATIENTS
PEDW_UNIQ.query('LOS>=@UPPER_BOUND').drop_duplicates(subset=['ALF_PE'], ignore_index=True).shape[0]

In [ ]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>=@UPPER_BOUND').shape[0]

In [ ]:
# obtain distribution of comorbidity counts per patients across all unique admissions with LOS>=@UPPER_BOUND
PEDW_UNIQ.query('LOS>=@UPPER_BOUND').NUM_ADMISSION_COMORBIDITIES.describe()

## ETHNICITY

In [468]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS>@UPPER_BOUND').drop_duplicates(subset=['ALF_PE'], ignore_index=True).ETHN_EC_ONS_DATE_LATEST_DESC.value_counts().to_csv("UBethn_patientcount.csv")

In [469]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>@UPPER_BOUND').ETHN_EC_ONS_DATE_LATEST_DESC.value_counts().to_csv("UBethn_count.csv")

In [470]:
PEDW_UNIQ.query('LOS>@UPPER_BOUND').groupby(['ETHN_EC_ONS_DATE_LATEST_DESC'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("UBNumcom_ethn.csv")

In [471]:
PEDW_UNIQ.query('LOS>@UPPER_BOUND').groupby(['ETHN_EC_ONS_DATE_LATEST_DESC'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("UBLOSethn.csv")

## GENDER

In [ ]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS>@UPPER_BOUND').drop_duplicates(subset=['ALF_PE'], ignore_index=True).GNDR_DESC.value_counts().to_csv("UBgndr_patientcount.csv")

In [ ]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>@UPPER_BOUND').GNDR_DESC.value_counts().to_csv("UBGNDR_count.csv")

In [ ]:
PEDW_UNIQ.query('LOS>@UPPER_BOUND').groupby(['GNDR_DESC'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("UBNumcom_gndr.csv")

In [ ]:
PEDW_UNIQ.query('LOS>@UPPER_BOUND').groupby(['GNDR_DESC'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("UBLOSgndr.csv")

## AGE GROUP

In [472]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS>@UPPER_BOUND').drop_duplicates(subset=['ALF_PE'], ignore_index=True).AGEGRP_AT_ADMIS_DT.value_counts().to_csv("UBagegrp_patientcount.csv")

In [473]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>@UPPER_BOUND').AGEGRP_AT_ADMIS_DT.value_counts().to_csv("UBagegrp_count.csv")

In [474]:
PEDW_UNIQ.query('LOS>@UPPER_BOUND').groupby(['AGEGRP_AT_ADMIS_DT'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("UBNumcom_agegrp.csv")

In [475]:
PEDW_UNIQ.query('LOS>@UPPER_BOUND').groupby(['AGEGRP_AT_ADMIS_DT'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("UBLOSagegrp.csv")

## WIMD

In [ ]:
#Number of uniqie patients
PEDW_UNIQ.query('LOS>@UPPER_BOUND').drop_duplicates(subset=['ALF_PE'], ignore_index=True).WIMD2019_QUINTILE_AT_INDEX_DATE.value_counts().to_csv("UBwimd_patientcount.csv")

In [476]:
#NUMBER OF UNIQUE ADMISSIONS
PEDW_UNIQ.query('LOS>@UPPER_BOUND').WIMD2019_QUINTILE_AT_INDEX_DATE.value_counts().to_csv("UBwimd_count.csv")

In [477]:
PEDW_UNIQ.query('LOS>@UPPER_BOUND').groupby(['WIMD2019_QUINTILE_AT_INDEX_DATE'])['NUM_ADMISSION_COMORBIDITIES'].agg(['mean', 'std']).reset_index().to_csv("UBNumcom_wimd.csv")

In [478]:
PEDW_UNIQ.query('LOS>@UPPER_BOUND').groupby(['WIMD2019_QUINTILE_AT_INDEX_DATE_DESC'])['LOS'].agg(['mean', 'std']).reset_index().to_csv("UBLOSwimd.csv")